In [1]:
!pip install tensorflow

In [2]:
#Definition of the model:

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Input
import numpy as np
import cv2
import os

# Define DnCNN model for color images
def build_dncnn_model(depth=17, filters=64, image_channels=3):
    input_layer = Input(shape=(None, None, image_channels))
    x = Conv2D(filters, (3, 3), padding='same', use_bias=False)(input_layer)
    x = ReLU()(x)
    
    # Middle layers
    for _ in range(depth - 2):
        x = Conv2D(filters, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
    # Output layer
    x = Conv2D(image_channels, (3, 3), padding='same', use_bias=False)(x)
    output_layer = tf.keras.layers.Subtract()([input_layer, x])
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Function to denoise an image using the model
def denoise_image(model, image):
    # Normalize and reshape the image for the model
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    # Perform denoising
    denoised_image = model.predict(image)
    # Rescale the denoised image to 0-255 and convert to uint8
    denoised_image = np.clip(denoised_image[0] * 255.0, 0, 255).astype('uint8')
    return denoised_image

# Function to denoise a dataset of images
def denoise_dataset(model, input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    for filename in os.listdir(input_dir):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
            denoised_image = denoise_image(model, image)
            denoised_image = cv2.cvtColor(denoised_image, cv2.COLOR_RGB2BGR)  # Convert back to BGR for saving
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, denoised_image)

# Load the model and its pretrained weights
dncnn = build_dncnn_model(image_channels=3)
# Load pretrained weights here, for example:
# dncnn.load_weights('dncnn_weights.h5')

# Example usage
input_directory = '/kaggle/input/random'  # Directory containing noisy underwater images
output_directory = '/kaggle/working/'
denoise_dataset(dncnn, input_directory, output_directory)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 

In [10]:
#Training the model: 

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Input
import numpy as np
import cv2
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define DnCNN model for color images
def build_dncnn_model(depth=17, filters=64, image_channels=3):
    input_layer = Input(shape=(None, None, image_channels))
    x = Conv2D(filters, (3, 3), padding='same', use_bias=False)(input_layer)
    x = ReLU()(x)
    
    # Middle layers
    for _ in range(depth - 2):
        x = Conv2D(filters, (3, 3), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
    # Output layer
    x = Conv2D(image_channels, (3, 3), padding='same', use_bias=False)(x)
    output_layer = tf.keras.layers.Subtract()([input_layer, x])
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Function to add synthetic noise to a color image
def add_noise(image, noise_factor=25):
    noise = np.random.normal(0, noise_factor, image.shape)
    noisy_image = image + noise
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image

# Function to generate batches of images for training
def data_generator(clean_images, batch_size=16, patch_size=40, noise_factor=25):
    while True:
        batch_input = []
        batch_output = []
        for _ in range(batch_size):
            # Randomly select an image
            image = clean_images[np.random.randint(0, len(clean_images))]
            # Randomly crop a patch
            h, w, _ = image.shape
            top = np.random.randint(0, h - patch_size)
            left = np.random.randint(0, w - patch_size)
            patch = image[top:top + patch_size, left:left + patch_size, :]
            # Add noise
            noisy_patch = add_noise(patch, noise_factor)
            # Normalize and add to batch
            patch = patch.astype('float32') / 255.0
            noisy_patch = noisy_patch.astype('float32') / 255.0
            batch_input.append(noisy_patch)
            batch_output.append(patch)
        yield np.array(batch_input), np.array(batch_output)

# Load clean images from a directory
def load_images_from_directory(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
            images.append(image)
    return images

# Train the DnCNN model
def train_dncnn(clean_image_directory, model_save_path, epochs=50, batch_size=16, steps_per_epoch=100):
    # Load clean images
    clean_images = load_images_from_directory(clean_image_directory)

    # Build the model
    dncnn = build_dncnn_model(image_channels=3)
    dncnn.compile(optimizer='adam', loss='mean_squared_error')

    # Create data generator
    train_generator = data_generator(clean_images, batch_size=batch_size)

    # Checkpoint to save the model weights
    checkpoint = ModelCheckpoint(model_save_path, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')

    # Train the model
    dncnn.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint])

    # Save the model weights manually after training (if needed)
    dncnn.save_weights(model_save_path)

# Example usage
clean_image_directory = '/kaggle/working/'  # Directory with clean training images
model_save_path = '/kaggle/working/dncnn_weights.weights.h5'  # Path to save the trained model weights
train_dncnn(clean_image_directory, model_save_path)

Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.1505
Epoch 1: loss improved from inf to 0.04058, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - loss: 0.1494
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0053
Epoch 2: loss improved from 0.04058 to 0.00514, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - loss: 0.0053
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0049
Epoch 3: loss improved from 0.00514 to 0.00482, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 0.0049
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0048
Epoch 4: loss improved from 0.00482 to 0.00468, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - loss: 0.0048
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0042
Epoch 5: loss improved fro

In [ ]:
#Evaluation:



In [11]:
!pip install fpdf matplotlib

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=abcdddde08c571bcd4b8f3fbd6d1ca06042fa7a16817d4c377dd861e80d73408
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [12]:
import matplotlib.pyplot as plt
from fpdf import FPDF
import re

# Sample log data
log_data = """
(Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.1505
Epoch 1: loss improved from inf to 0.04058, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - loss: 0.1494
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0053
Epoch 2: loss improved from 0.04058 to 0.00514, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - loss: 0.0053
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0049
Epoch 3: loss improved from 0.00514 to 0.00482, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 0.0049
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0048
Epoch 4: loss improved from 0.00482 to 0.00468, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - loss: 0.0048
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0042
Epoch 5: loss improved from 0.00468 to 0.00410, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - loss: 0.0042
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0036
Epoch 6: loss improved from 0.00410 to 0.00358, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 0.0036
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0032
Epoch 7: loss improved from 0.00358 to 0.00303, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 0.0032
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0026
Epoch 8: loss improved from 0.00303 to 0.00253, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 0.0026
Epoch 9/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0023
Epoch 9: loss improved from 0.00253 to 0.00232, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.0023
Epoch 10/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0022
Epoch 10: loss improved from 0.00232 to 0.00224, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 0.0022
Epoch 11/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0020
Epoch 11: loss improved from 0.00224 to 0.00200, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.0020
Epoch 12/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0019
Epoch 12: loss improved from 0.00200 to 0.00185, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 0.0019
Epoch 13/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0019
Epoch 13: loss improved from 0.00185 to 0.00182, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.0019
Epoch 14/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0017
Epoch 14: loss improved from 0.00182 to 0.00169, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.0017
Epoch 15/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0016
Epoch 15: loss improved from 0.00169 to 0.00159, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 0.0016
Epoch 16/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0016
Epoch 16: loss improved from 0.00159 to 0.00156, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - loss: 0.0016
Epoch 17/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0015
Epoch 17: loss improved from 0.00156 to 0.00152, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 0.0015
Epoch 18/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0014
Epoch 18: loss improved from 0.00152 to 0.00138, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 0.0014
Epoch 19/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0013
Epoch 19: loss improved from 0.00138 to 0.00134, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 0.0013
Epoch 20/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0014
Epoch 20: loss improved from 0.00134 to 0.00131, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - loss: 0.0014
Epoch 21/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0012
Epoch 21: loss improved from 0.00131 to 0.00121, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - loss: 0.0012
Epoch 22/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0012
Epoch 22: loss did not improve from 0.00121
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 0.0012
Epoch 23/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0012
Epoch 23: loss improved from 0.00121 to 0.00114, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 0.0012
Epoch 24/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0011
Epoch 24: loss improved from 0.00114 to 0.00110, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 0.0011
Epoch 25/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0012
Epoch 25: loss improved from 0.00110 to 0.00108, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 0.0012
Epoch 26/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0010
Epoch 26: loss improved from 0.00108 to 0.00101, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 0.0010
Epoch 27/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 9.9572e-04
Epoch 27: loss improved from 0.00101 to 0.00099, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 9.9562e-04
Epoch 28/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0011
Epoch 28: loss did not improve from 0.00099
100/100 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - loss: 0.0011
Epoch 29/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 9.1565e-04
Epoch 29: loss improved from 0.00099 to 0.00094, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 9.1584e-04
Epoch 30/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 9.5681e-04
Epoch 30: loss improved from 0.00094 to 0.00092, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 9.5648e-04
Epoch 31/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.7967e-04
Epoch 31: loss did not improve from 0.00092
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 8.8040e-04
Epoch 32/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.0011
Epoch 32: loss did not improve from 0.00092
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 0.0011
Epoch 33/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.5740e-04
Epoch 33: loss improved from 0.00092 to 0.00084, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 8.5719e-04
Epoch 34/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.2548e-04
Epoch 34: loss improved from 0.00084 to 0.00082, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - loss: 8.2540e-04
Epoch 35/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.2493e-04
Epoch 35: loss improved from 0.00082 to 0.00081, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - loss: 8.2482e-04
Epoch 36/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 7.7121e-04
Epoch 36: loss improved from 0.00081 to 0.00077, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 7.7117e-04
Epoch 37/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 7.3647e-04
Epoch 37: loss improved from 0.00077 to 0.00073, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - loss: 7.3645e-04
Epoch 38/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 9.2881e-04
Epoch 38: loss did not improve from 0.00073
100/100 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - loss: 9.2853e-04
Epoch 39/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 7.6532e-04
Epoch 39: loss did not improve from 0.00073
100/100 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - loss: 7.6571e-04
Epoch 40/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 7.5704e-04
Epoch 40: loss did not improve from 0.00073
100/100 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - loss: 7.5723e-04
Epoch 41/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 7.4464e-04
Epoch 41: loss improved from 0.00073 to 0.00070, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - loss: 7.4420e-04
Epoch 42/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 6.8622e-04
Epoch 42: loss improved from 0.00070 to 0.00069, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - loss: 6.8627e-04
Epoch 43/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 6.3716e-04
Epoch 43: loss improved from 0.00069 to 0.00065, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - loss: 6.3731e-04
Epoch 44/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.8828e-04
Epoch 44: loss improved from 0.00065 to 0.00059, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 5.8827e-04
Epoch 45/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.8991e-04
Epoch 45: loss did not improve from 0.00059
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 5.9003e-04
Epoch 46/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.6566e-04
Epoch 46: loss improved from 0.00059 to 0.00057, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - loss: 5.6568e-04
Epoch 47/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.8021e-04
Epoch 47: loss did not improve from 0.00057
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - loss: 5.8028e-04
Epoch 48/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.6365e-04
Epoch 48: loss improved from 0.00057 to 0.00053, saving model to /kaggle/working/dncnn_weights.weights.h5
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - loss: 5.6334e-04
Epoch 49/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.8511e-04
Epoch 49: loss did not improve from 0.00053
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 8.8493e-04
Epoch 50/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 6.0923e-04
Epoch 50: loss did not improve from 0.00053
100/100 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - loss: 6.0954e-04
)
"""

# Regular expression pattern to match the loss values
pattern = r'loss: ([0-9.e-]+)'

# Extract all loss values
losses = re.findall(pattern, log_data)

# Convert to float
losses = [float(loss) for loss in losses]

# Function to generate PDF report
def generate_pdf_report(logs, pdf_path):
    # Plot the loss over epochs
    plt.plot(logs, label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plot_path = '/kaggle/working/loss_plot.png'
    plt.savefig(plot_path)
    plt.close()

    # Create a PDF report
    pdf = FPDF()
    pdf.add_page()
    
    # Title
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, 'Training Report', 0, 1, 'C')
    
    # Loss Plot
    pdf.set_font('Arial', '', 12)
    pdf.cell(0, 10, 'Loss over Epochs', 0, 1, 'L')
    pdf.image(plot_path, x=None, y=None, w=180)
    
    # Save the PDF
    pdf.output(pdf_path)

# Path to save the training report
report_path = '/kaggle/working/training_report.pdf'

# Generate PDF report using the extracted logs
generate_pdf_report(losses, report_path)
